In [ ]:
import pandas as pd
import numpy as np
import math as mt

from rich.console import Console
from rich.table import Table 

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from sklearn.decomposition import PCA

from sklearn import metrics
from sklearn.cluster import KMeans
from sklearn.cluster import DBSCAN
from sklearn.cluster import AgglomerativeClustering

from sklearn.metrics import silhouette_score

In [ ]:
from dataframe.cores import *
# Declara a classe cores
cores = cores()

In [ ]:
from IPython.display import display

In [ ]:
df = pd.read_csv('Resposta.csv', sep=',')

In [ ]:
display(df)

---

# Função de Otimização do KMeans

Define uma função ideal para o kmeans que executa clustering K-means em um determinado DataFrame df para um intervalo de números de cluster e visualiza os resultados. A função leva três argumentos: df (o DataFrame a ser agrupado), start (o número inicial de clusters a testar) e end (o número final de clusters a testar). Resultando em todos os possiveis valores de clusters e suas respectivas inércias.

In [ ]:
# Esta função plota a soma dos erros quadráticos (SSE) e o escore de silhueta para um intervalo de clusters.
def optimal_kmeans(df, start=2, end=11):
    
    '''
    df: DataFrame que sera clusterizado.
    start: Número inicial de clusters que você deseja testar.
    end: Número final de clusters que você deseja testar.
    '''
    
    # Soma dos erros quadráticos
    sse = {}
    
    # Escore de silhueta
    silhouette = {}
    
    # Adiciona um dicionario de resultados
    results = {}
    
    for k in range(start, end):
        kmeans = KMeans(n_clusters=k, max_iter=100, random_state=42).fit(df)
        df["clusters"] = kmeans.labels_
        sse[k] = kmeans.inertia_ 
        silhouette[k] = silhouette_score(df, kmeans.labels_, metric='euclidean')
      

    # Plota um subplot com o plotly com o primeiro gráfico sendo o sse e o outro com o silouette
    fig = make_subplots(rows=1, cols=2, subplot_titles=("SSE", "Silhouette Score"))

    fig.add_trace(go.Scatter(x=list(sse.keys()), y=list(sse.values()), 
                             name='SSE'), row=1, col=1)
    
    fig.add_trace(go.Scatter(x=list(silhouette.keys()), y=list(silhouette.values()), 
                             name='Silhouette Score'), row=1, col=2)
    
    fig.update_layout(title_text="Gráfico de análise - SSE e Silhouette Score")

    fig.update_layout(
        autosize=False,
        width=1800,
        height=800,
    )    

    fig.show()
    
    # Com o rich, cria uma tabela com os resultados
    table = Table(title="Resultados")
    table.add_column("Clusters", justify="center", style="cyan", no_wrap=True)
    table.add_column("SSE", justify="center", style="magenta", no_wrap=True)
    table.add_column("Silhouette Score", justify="center", style="green", no_wrap=True)
    
    for k in range(start, end):
        table.add_row(str(k), str(sse[k]), str(silhouette[k]))

    console = Console()
    console.print(table)

# Uso da função
optimal_kmeans(df)

# Função de Otimização do DBSCAN

Define uma função ideal para o dbscan que é usada para encontrar os parâmetros ideais para o algoritmo de cluster DBSCAN (Density-Based Spatial Clustering of Applications with Noise). A função leva um DataFrame df para ser agrupado, um intervalo de eps_values (distância máxima entre duas amostras para que elas sejam consideradas na mesma vizinhança) e min_samples_values (o número de amostras em uma vizinhança para que um ponto seja considerado como um ponto central) como entradas. Realizando uma busca em grade, mostrando todos os valores possiveis

In [ ]:
# Esta função plota o escore de silhueta para uma variedade de valores eps e min_samples.
def optimal_dbscan(df, eps_values=np.linspace(0.1, 20, 50), min_samples_values=range(2, 15)):
    
    '''
    df: DataFrame que sera clusterizado.
    eps_values: Valores de eps que você deseja testar.
    min_samples_values: Valores de min_samples que você deseja testar.
    '''
    
    # Escore de silhueta
    silhouette = {}
    
    for eps in eps_values:
        for min_samples in min_samples_values:
            dbscan = DBSCAN(eps=eps, min_samples=min_samples).fit(df)
            n_clusters = len(set(dbscan.labels_))
            if 1 < n_clusters < df.shape[0]:  # Deve haver pelo menos 2 clusters e menos que n_samples para o escore de silhueta
                silhouette[(eps, min_samples)] = silhouette_score(df, dbscan.labels_)
    
    # Plota um gráfico com o plotly
    fig = go.Figure(data=go.Scatter(x=[str(i) for i in silhouette.keys()], y=list(silhouette.values())))
    
    fig.update_layout(title_text="Gráfico de análise - DBSCAN")

    fig.update_layout(
        autosize=False,
        width=1400,
        height=1000,
    )
    
    fig.show()
    
    # Com o rich, cria uma tabela com os resultados
    table = Table(title="Resultados")
    table.add_column("Eps", justify="center", style="cyan", no_wrap=True)
    table.add_column("Min Samples", justify="center", style="magenta", no_wrap=True)
    table.add_column("Silhouette Score", justify="center", style="green", no_wrap=True)
    
    for key, value in silhouette.items():
        table.add_row(str(key[0]), str(key[1]), str(value))
    
    console = Console()
    console.print(table)
    
optimal_dbscan(df)

# Função de Otimização do AGNES

Define a função que é usada para encontrar o número ideal de clusters e método de ligação para Agglomerative Clustering (AGNES) em um determinado conjunto de dados. AGNES é um tipo de método de agrupamento hierárquico que mescla o par mais próximo de clusters em cada etapa. Resultando em todos os possiveis valores de clusters e suas respectivas inércias.

In [ ]:
def optimal_agnes(df, start=2, end=11, linkage_methods=['ward', 'complete', 'average', 'single']):
    '''
    Esta função plota o escore de silhueta para um intervalo de números de clusters e métodos de ligação.
    df: DataFrame. Seus dados.
    start: int. Número inicial de clusters que você deseja testar.
    end: int. Número final de clusters que você deseja testar.
    linkage_methods: list. Métodos de ligação que você deseja testar.
    '''
    
    # Escore de silhueta
    silhouette = {}
    
    for linkage in linkage_methods:
        for n_clusters in range(start, end):
            agnes = AgglomerativeClustering(n_clusters=n_clusters, linkage=linkage).fit(df)
            labels = agnes.labels_
            silhouette[(linkage, n_clusters)] = silhouette_score(df, labels, metric='euclidean')
    
    # Plota um gráfico com o plotly com todos os métodos de ligação na mesma figura
    fig = go.Figure()

    for linkage in linkage_methods:
        fig.add_trace(go.Scatter(x=list(range(start, end)), y=[silhouette[(linkage, i)] for i in range(start, end)], 
                         mode='lines+markers', name=linkage))
        
    fig.update_layout(title_text="Gráfico de análise - Agglomerative Clustering")

    fig.update_layout(
        autosize=False,
        width=1400,
        height=800,
    )

    fig.show()

    # Com o rich, cria uma tabela com os resultados
    table = Table(title="Resultados")
    table.add_column("Linkage", justify="center", style="cyan", no_wrap=True)
    table.add_column("Clusters", justify="center", style="magenta", no_wrap=True)
    table.add_column("Silhouette Score", justify="center", style="green", no_wrap=True)
    
    for key, value in silhouette.items():
        table.add_row(str(key[0]), str(key[1]), str(value))
        
    console = Console()
    console.print(table)

# Uso da função
optimal_agnes(df)

---

# Técnicas utilizadas para redução de dimensionalidade

## PCA (Principal Component Analysis)

PCA, ou Análise de Componentes Principais, é uma técnica de redução de dimensionalidade usada em aprendizado de máquina e estatística. O objetivo do PCA é transformar um conjunto de variáveis possivelmente correlacionadas em um conjunto menor de variáveis não correlacionadas chamadas componentes principais.

### No código

```python

pca = PCA(n_components=2)

```
- n_components: número de componentes principais a serem mantidos, aqui, um objeto PCA é criado, especificando que queremos reduzir nosso conjunto de dados para 2 componentes principais

```python
principalComponents = pca.fit_transform(df)
```

- O método fit_transform() é chamado no objeto PCA, que calcula os componentes principais do DataFrame df e usa esses componentes para transformar df em um novo espaço de dados. O resultado é um array numpy principalComponents que contém os dados transformados



---

# Método do KMeans

O KMeans é um método de clusterização que tem como objetivo dividir um conjunto de dados em K grupos, onde K é um número pré-definido

In [ ]:
# Define a função para o KMeans
def KMeans_Trainer(df):
    
    # Determina o número de clusters do K-Means
    n_clusters = 5

    # Determina o máximo de iteração
    max_iter = 100
    
    # Define o modelo
    model = KMeans(n_clusters=n_clusters, max_iter=max_iter, random_state=42)    
    
    # Define os dados de treino
    model.fit(df)
    
    # Define a opinião do modelo
    X = model.labels_

    # Printa a opinião do modelo com o rich
    console = Console()
    table = Table(title="Opinião do Modelo K-Means")
    table.add_column("Cluster", style="cyan", justify="center")
    table.add_column("Opinião", style="magenta", justify="center")
    for i in range(n_clusters):
        table.add_row(str(i), str(np.sum(X == i)))
    console.print(table)
    
    # Printa o X com o rich
    console = Console()
    console.print(X)
    
    # Plota o gráfico de dispersão dos dados K-Means
    
    # Supondo que meu dataframe seja o 'X' e que 'y_kmeans' seja o resultado do agrupamento
    pca = PCA(n_components=2)
    principalComponents = pca.fit_transform(df)

    # Cria um dataframe com os componentes principais
    principalDf = pd.DataFrame(data = principalComponents, columns = ['principal component 1', 'principal component 2'])

    # Adiciona a coluna de agrupamento
    principalDf['cluster'] = X

    # Ordena o DataFrame pelo valor do cluster (Arruma a legenda do gráfico para que os clusters fiquem em ordem crescente)
    principalDf = principalDf.sort_values(by='cluster')

    # Transforma na df todos o cluster, os que era 0 viram 1, 1 vira 2 e assim por diante
    principalDf['cluster'] = principalDf['cluster'] + 1

    # Plota o gráfico de dispersão
    fig = px.scatter(principalDf, x='principal component 1', y='principal component 2', symbol='cluster', size='cluster', color='cluster')

    # Atualiza o layout do gráfico
    fig.update_layout(
        width=800, 
        height=800)

    # Atualiza o X e Y do gráfico retirando-os
    fig.update_xaxes(title_text='')
    fig.update_yaxes(title_text='')
    
    # Remove a barra de cores
    fig.update_layout(coloraxis_showscale=False)
    
    # Retira o fundo do gráfico (grid) colocando-o branco
    fig.update_layout(plot_bgcolor='white')
    
    # Retira o axis do gráfico
    fig.update_xaxes(showline=False, showgrid=False, zeroline=False)
    fig.update_yaxes(showline=False, showgrid=False, zeroline=False)
    
    # Retira os numeros do eixo X e Y
    fig.update_xaxes(showticklabels=False)
    fig.update_yaxes(showticklabels=False)
    
    # Organiza a legenda em ordem crescente de clusters
    fig.for_each_trace(lambda t: t.update(name='Cluster ' + str(t.name)))

    # Configura a legenda
    fig.update_layout(
        legend=dict(
            traceorder='normal',
            font=dict(
                family='sans-serif',
                size=20,
                color='black'
            ),
            bordercolor='Black',
            borderwidth=2
        ),
        
        # Padding para tirar o espaço em branco em volta do gráfico
        margin=dict(l=0, r=0, t=0, b=0)
    )
    
    # Retira o titulo da legenda
    fig.update_layout(legend_title_text='')

    # Exibe o gráfico
    fig.show()
    
    # Soma dos quadrados das distâncias
    sse = model.inertia_
    
    # Calcula a coesao
    cohesion = mt.sqrt(model.inertia_)/model.n_clusters
    
    # Calcula o coeficiente de silhueta
    silhouette_kmeans = silhouette_score(df, X)
    
    # Rand score do KMeans
    rand_kmeans = metrics.adjusted_rand_score(df['clusters'], X)
    
    # Homogeneidade
    homogeneity = metrics.homogeneity_score(df['clusters'], X)
    
    # Completude
    completeness = metrics.completeness_score(df['clusters'], X)
    
    # Matriz de confusão
    confusion_matrix = metrics.cluster.contingency_matrix(df['clusters'], X)
    
    
    # Printa 
    console = Console()
    table = Table(title="Resultados do K-Means")
    table.add_column("SSE", style="cyan", justify="center")
    table.add_column("Coesão", style="magenta", justify="center")
    table.add_column("Silhouette Score", style="green", justify="center")
    table.add_column("Homogeneidade", style="blue", justify="center")
    table.add_column("Rand Score", style="yellow", justify="center")
    table.add_column("Completude", style="red", justify="center")
    
    
    table.add_row(str(sse), str(cohesion), str(silhouette_kmeans), str(homogeneity), str(rand_kmeans), str(completeness))
    
    console.print(table)

    # Printa a matriz de confusão
    print("Matriz de Confusão")
    print(confusion_matrix)
    
    
    return silhouette_kmeans
    
# Chamada da função KMeans
silhouette_kmeans = KMeans_Trainer(df)


# Método do DBSCAN

O DBSCAN é um método de clusterização que tem como objetivo dividir um conjunto de dados em grupos de densidade, onde os grupos são formados por pontos que estão próximos uns dos outros

In [ ]:
def DBSCAN_Trainer(df):
    
    # Define o EPS
    eps = 7
    
    # Define o número mínimo de amostras
    min_samples = 5
    
    # Define o modelo
    model = DBSCAN(eps=eps, min_samples=min_samples)
    
    # Define os dados de treino
    model.fit(df)
    
    # Define a opinião do modelo
    X = model.labels_

    # Printa a opinião do modelo com o rich
    console = Console()
    table = Table(title="Opinião do Modelo DBSCAN")
    table.add_column("Cluster", style="cyan", justify="center")
    table.add_column("Opinião", style="magenta", justify="center")
    for i in range(len(np.unique(X))):
        table.add_row(str(i), str(np.sum(X == i)))
    console.print(table)
    
    # Printa o X com o rich
    console = Console()
    console.print(X)
    
    # Plota o gráfico de dispersão dos dados DBSCAN
    
    # Supondo que meu dataframe seja o 'X' e que 'y_kmeans' seja o resultado do agrupamento
    pca = PCA(n_components=2)
    principalComponents = pca.fit_transform(df)

    # Cria um dataframe com os componentes principais
    principalDf = pd.DataFrame(data = principalComponents, columns = ['principal component 1', 'principal component 2'])

    # Adiciona a coluna de agrupamento
    principalDf['cluster'] = X

    # Ordena o DataFrame pelo valor do cluster (Arruma a legenda do gráfico para que os clusters fiquem em ordem crescente)
    principalDf = principalDf.sort_values(by='cluster')

    # Transforma na df todos o cluster, os que era 0 viram 1, 1 vira 2 e assim por diante
    principalDf['cluster'] = principalDf['cluster'] + 1

    
    # Plota o gráfico de dispersão
    fig = px.scatter(principalDf, x='principal component 1', y='principal component 2', symbol='cluster', size='cluster', color='cluster')

    # Atualiza o layout do gráfico
    fig.update_layout(
        width=800, 
        height=800)
    
    # Atualiza o X e Y do gráfico retirando-os
    fig.update_xaxes(title_text='')
    fig.update_yaxes(title_text='')
    
    # Remove a barra de cores
    fig.update_layout(coloraxis_showscale=False)
    
    # Retira o fundo do gráfico (grid) colocando-o branco
    fig.update_layout(plot_bgcolor='white')

    # Retira o axis do gráfico
    fig.update_xaxes(showline=False, showgrid=False, zeroline=False)
    fig.update_yaxes(showline=False, showgrid=False, zeroline=False)
    
    # Retira os numeros do eixo X e Y
    fig.update_xaxes(showticklabels=False)
    fig.update_yaxes(showticklabels=False)
    
    # Organiza a legenda em ordem crescente de clusters
    fig.for_each_trace(lambda t: t.update(name='Cluster ' + str(t.name)))
    
    # Configura a legenda
    fig.update_layout(
        legend=dict(
            traceorder='normal',
            font=dict(
                family='sans-serif',
                size=20,
                color='black'
            ),
            bordercolor='Black',
            borderwidth=2
        ),
        
        # Padding para tirar o espaço em branco em volta do gráfico
        margin=dict(l=0, r=0, t=0, b=0)
    )
    
    # Retira o titulo da legenda
    fig.update_layout(legend_title_text='')
    
    # Exibe o gráfico
    fig.show()
    
    # Calcula o coeficiente de silhueta
    n_clusters = len(set(X))
    
    if n_clusters > 1:
        silhouette_dbscan = silhouette_score(df, X)
    else:
        silhouette_dbscan = 0
    
    
    # Rand score do DBSCAN
    rand_dbscan = metrics.adjusted_rand_score(df['clusters'], X)
    
    # Homogeneidade
    homogeneity = metrics.homogeneity_score(df['clusters'], X)
    
    # Completude
    completeness = metrics.completeness_score(df['clusters'], X)
    
    # Matriz de confusão
    confusion_matrix = metrics.cluster.contingency_matrix(df['clusters'], X)
    
    # Printa
    console = Console()
    table = Table(title="Resultados do DBSCAN")
    table.add_column("Silhouette Score", style="cyan", justify="center")
    table.add_column("Rand Score", style="magenta", justify="center")
    table.add_column("Homogeneidade", style="green", justify="center")
    table.add_column("Completude", style="yellow", justify="center")
    
    table.add_row(str(silhouette_dbscan), str(rand_dbscan), str(homogeneity), str(completeness))
    
    console.print(table)
    
    # Printa a matriz de confusão
    print("Matriz de Confusão")
    print(confusion_matrix)
    
    return silhouette_dbscan

# Chamada da função DBSCAN
silhouette_dbscan = DBSCAN_Trainer(df)

# Método do AGNES

O AGNES é um método de clusterização que tem como objetivo dividir um conjunto de dados em grupos de hierarquia, onde os grupos são formados por pontos que estão próximos uns dos outros

In [ ]:
def AGNES_Trainer(df):
    
    # Define o número de clusters
    n_clusters = 5
    
    # Define o método de ligação
    linkage = 'average'
    
    # Define o modelo
    model = AgglomerativeClustering(n_clusters=n_clusters, linkage=linkage)
    
    # Define os dados de treino
    model.fit(df)
    
    # Define a opinião do modelo
    X = model.labels_

    # Printa a opinião do modelo com o rich
    console = Console()
    table = Table(title="Opinião do Modelo AGNES")
    table.add_column("Cluster", style="cyan", justify="center")
    table.add_column("Opinião", style="magenta", justify="center")
    for i in range(n_clusters):
        table.add_row(str(i), str(np.sum(X == i)))
    console.print(table)
    
    # Printa o X com o rich
    console = Console()
    console.print(X)
    
    # Plota o gráfico de dispersão dos dados AGNES
    
    # Supondo que meu dataframe seja o 'X' e que 'y_kmeans' seja o resultado do agrupamento
    pca = PCA(n_components=2)
    principalComponents = pca.fit_transform(df)

    # Cria um dataframe com os componentes principais
    principalDf = pd.DataFrame(data = principalComponents, columns = ['principal component 1', 'principal component 2'])

    # Adiciona a coluna de agrupamento
    principalDf['cluster'] = X

    # Ordena o DataFrame pelo valor do cluster (Arruma a legenda do gráfico para que os clusters fiquem em ordem crescente)
    principalDf = principalDf.sort_values(by='cluster')
    
    # Transforma na df todos o cluster, os que era 0 viram 1, 1 vira 2 e assim por diante
    principalDf['cluster'] = principalDf['cluster'] + 1

    # Plota o gráfico de dispersão usando o matplotlib
    fig = px.scatter(principalDf, x='principal component 1', y='principal component 2', symbol='cluster', size='cluster', color='cluster')
    
    # Atualiza o layout do gráfico
    fig.update_layout(
        width=800, 
        height=800)
    
    # Atualiza o X e Y do gráfico retirando-os
    fig.update_xaxes(title_text='')
    fig.update_yaxes(title_text='')
    
    # Remove a barra de cores
    fig.update_layout(coloraxis_showscale=False)
    
    # Retira o fundo do gráfico (grid) colocando-o branco
    fig.update_layout(plot_bgcolor='white')

    # Retira o axis do gráfico
    fig.update_xaxes(showline=False, showgrid=False, zeroline=False)   
    fig.update_yaxes(showline=False, showgrid=False, zeroline=False)
    
    # Retira os numeros do eixo X e Y
    fig.update_xaxes(showticklabels=False)
    fig.update_yaxes(showticklabels=False)
    
    # Organiza a legenda em ordem crescente de clusters
    fig.for_each_trace(lambda t: t.update(name='Cluster ' + str(t.name)))
    
    # Configura a legenda
    fig.update_layout(
        legend=dict(
            traceorder='normal',
            font=dict(
                family='sans-serif',
                size=20,
                color='black'
            ),
            bordercolor='Black',
            borderwidth=2
        ),
        
        # Padding para tirar o espaço em branco em volta do gráfico
        margin=dict(l=0, r=0, t=0, b=0)
    )
    
    # Retira o titulo da legenda
    fig.update_layout(legend_title_text='')
    
    # Exibe o gráfico
    fig.show()
    
    # Calcula o coeficiente de silhueta
    silhouette_agnes = silhouette_score(df, X)
    
    # Rand score do AGNES
    rand_agnes = metrics.adjusted_rand_score(df['clusters'], X)
    
    # Homogeneidade
    homogeneity = metrics.homogeneity_score(df['clusters'], X)
    
    # Completude
    completeness = metrics.completeness_score(df['clusters'], X)
    
    # Matriz de confusão
    confusion_matrix = metrics.cluster.contingency_matrix(df['clusters'], X)
    
    # Printa
    console = Console()
    table = Table(title="Resultados do AGNES")
    table.add_column("Silhouette Score", style="cyan", justify="center")
    table.add_column("Rand Score", style="magenta", justify="center")
    table.add_column("Homogeneidade", style="green", justify="center")
    table.add_column("Completude", style="yellow", justify="center")
    
    table.add_row(str(silhouette_agnes), str(rand_agnes), str(homogeneity), str(completeness))
    
    console.print(table)
    
    # Printa a matriz de confusão
    print("Matriz de Confusão")
    print(confusion_matrix)
    
    return silhouette_agnes
    
# Chamada da função AGNES
silhouette_agnes = AGNES_Trainer(df)

---

# Técnicas extras

## O que são?

Tanto o t-SNE (t-Distributed Stochastic Neighbor Embedding) quanto o UMAP (Uniform Manifold Approximation and Projection) são técnicas de redução de dimensionalidade que são comumente usadas para visualização de dados de alta dimensão. Ambos seguem um processo semelhante onde começam calculando probabilidades de alta dimensão p, depois probabilidades de baixa dimensão q, seguido pelo cálculo da função de custo C(p,q) comparando as diferenças entre as probabilidade

No entanto, existem algumas diferenças importantes entre os dois:

- Velocidade e Escalabilidade: O UMAP geralmente é mais rápido que o t-SNE e pode lidar melhor com conjuntos de dados maiores.
- Preservação da Estrutura Global: Enquanto o t-SNE se concentra em preservar a estrutura local dos dados (ou seja, mantém pontos que são próximos no espaço de alta dimensão próximos no espaço de baixa dimensão), o UMAP tenta preservar tanto a estrutura local quanto a global dos dados. Isso significa que o UMAP pode preservar melhor as relações de distância entre os clusters.
- Reprodutibilidade: O t-SNE é uma técnica estocástica, o que significa que você pode obter resultados ligeiramente diferentes cada vez que você executar o código, mesmo com a mesma semente aleatória. Por outro lado, o UMAP é mais consistente e produz resultados mais reprodutíveis.
- Uso de Hiperparâmetros: Ambos os métodos têm hiperparâmetros que podem ser ajustados para alterar o resultado da redução de dimensionalidade. No entanto, a interpretação e o ajuste desses hiperparâmetros podem ser menos intuitivos no t-SNE do que no UMAP.

In [ ]:
from sklearn.manifold import TSNE

def TSNE_trainer(df):
    
    # Define o modelo
    model = TSNE(n_components=2, random_state=0)
    
    # Define os dados de treino
    X = model.fit_transform(df)
    
    # Plota o gráfico de dispersão dos dados TSNE
    
    # Cria um dataframe com os componentes principais
    principalDf = pd.DataFrame(data = X, columns = ['TSNE 1', 'TSNE 2'])

    # Plota o gráfico de dispersão
    fig = px.scatter(principalDf, x='TSNE 1', y='TSNE 2')

    # Atualiza o layout do gráfico
    fig.update_layout(
        width=800, 
        height=800)

    # Atualiza o X e Y do gráfico
    fig.update_xaxes(title_text='TSNE 1')
    fig.update_yaxes(title_text='TSNE 2')
    
    # Exibe o gráfico
    fig.show()
    
# Chamada da função TSNE
TSNE_trainer(df)

In [ ]:
from umap import UMAP

def UMAP_trainer(df):
    
    # Define o modelo
    model = UMAP(n_components=2, random_state=0)
    
    # Define os dados de treino
    X = model.fit_transform(df)
    
    # Plota o gráfico de dispersão dos dados UMAP
    
    # Cria um dataframe com os componentes principais
    principalDf = pd.DataFrame(data = X, columns = ['UMAP 1', 'UMAP 2'])

    # Plota o gráfico de dispersão
    fig = px.scatter(principalDf, x='UMAP 1', y='UMAP 2')

    # Atualiza o layout do gráfico
    fig.update_layout(
        width=800, 
        height=800)

    # Atualiza o X e Y do gráfico
    fig.update_xaxes(title_text='UMAP 1')
    fig.update_yaxes(title_text='UMAP 2')
    
    # Exibe o gráfico
    fig.show()
    
# Chamada da função UMAP
UMAP_trainer(df)